In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pdfplumber

with pdfplumber.open('./source-doc.pdf') as pdf:
    text = ""
    for page in pdf.pages:
        text += page.extract_text()

# Print the extracted text
#print(text)


In [3]:
cleaned_text = text.replace('\n', ' ')


In [4]:
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_text = tokenizer.tokenize(cleaned_text)

# Print the tokenized text
#print(tokenized_text)
with open('processed_data.txt', 'w') as file:
    file.write(" ".join(tokenized_text))


In [7]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Load the model
#model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

#from transformers import DistilBertForQuestionAnswering

# Load the pre-trained DistilBERT model
#model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

# Modify the classification head
#model.classifier = SomeCustomClassifier()


# Set the model in training mode
#model.train()


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [9]:
#model.config.hidden_size

In [12]:
"""
import torch
import torch.nn as nn
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer

# Define a custom classification head
class CustomClassifier(nn.Module):
    def __init__(self, hidden_size):
        super(CustomClassifier, self).__init__()
        self.start_classifier = nn.Linear(hidden_size, 1)
        self.end_classifier = nn.Linear(hidden_size, 1)

    def forward(self, hidden_states):
        start_logits = self.start_classifier(hidden_states).squeeze(-1)
        end_logits = self.end_classifier(hidden_states).squeeze(-1)
        return start_logits, end_logits
        
        
        """


In [ ]:
"""
import torch
import torch.nn as nn
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer

# Define a custom classification head with attention
class CustomClassifierWithAttention(nn.Module):
    def __init__(self, hidden_size):
        super(CustomClassifierWithAttention, self).__init__()
        self.attention = nn.MultiheadAttention(hidden_size, num_heads=1)
        self.start_classifier = nn.Linear(hidden_size, 1)
        self.end_classifier = nn.Linear(hidden_size, 1)

    def forward(self, hidden_states):
        # Apply self-attention
        attention_output, _ = self.attention(hidden_states, hidden_states, hidden_states)

        # Predict start and end positions
        start_logits = self.start_classifier(attention_output).squeeze(-1)
        end_logits = self.end_classifier(attention_output).squeeze(-1)

        return start_logits, end_logits

# Load the pre-trained DistilBERT model and tokenizer
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Replace the classification head with the custom classifier with attention
model.classifier = CustomClassifierWithAttention(model.config.hidden_size)

# Set the model in training mode
#model.train()
"""

In [21]:
import torch
import torch.nn as nn
from transformers import DistilBertForQuestionAnswering, DistilBertTokenizer, BertModel

# Define a custom classification head with contextual embeddings
class CustomClassifierWithContextualEmbeddings(nn.Module):
    def __init__(self, hidden_size):
        super(CustomClassifierWithContextualEmbeddings, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.start_classifier = nn.Linear(hidden_size, 1)
        self.end_classifier = nn.Linear(hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        # Get contextual embeddings from BERT
        bert_outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = bert_outputs.last_hidden_state

        # Predict start and end positions
        start_logits = self.start_classifier(hidden_states).squeeze(-1)
        end_logits = self.end_classifier(hidden_states).squeeze(-1)

        return start_logits, end_logits

# Load the pre-trained DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Create input tensors from your data (example)
#question = "What is the difference between innovation and invention?"
#context = "Innovation is validated and proven to be valuable in the marketplace, while invention is not validated and has no value other than theoretical value. Innovation can be a renewed method, process, business model, or product validated to create new value compared to previous solutions. On the other hand, invention is not proven in the marketplace and has no value other than theoretical value. The validation is a key factor that separates invention from innovation. In short, innovation is about providing a new value-generating product, service, or business model to validated markets, while invention is just an idea or a theoretical concept."
#encoded_inputs = tokenizer(question, context, truncation=True, padding=True, return_tensors='pt')
#input_ids = encoded_inputs['input_ids']
#attention_mask = encoded_inputs['attention_mask']

# Instantiate the custom classifier with contextual embeddings
model = CustomClassifierWithContextualEmbeddings(hidden_size=768)  # Assuming BERT-base

# Set the model in training mode
#model.train()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
# Freeze the BERT layers
for param in model.bert.parameters():
    param.requires_grad = False


In [43]:
# Define the data augmentation function
def augment_data(question, context):
    # Perform data augmentation here (example)
    augmented_question = question + " ?"
    augmented_context = context + " [MASK]"
    return augmented_question, augmented_context

# Create input tensors from your data (example)
#question = "What is the capital of France?"
#context = "Paris is the capital and largest city of France."
question = "What is the difference between innovation and invention?"
context = "Innovation is validated and proven to be valuable in the marketplace, while invention is not validated and has no value other than theoretical value. Innovation can be a renewed method, process, business model, or product validated to create new value compared to previous solutions. On the other hand, invention is not proven in the marketplace and has no value other than theoretical value. The validation is a key factor that separates invention from innovation. In short, innovation is about providing a new value-generating product, service, or business model to validated markets, while invention is just an idea or a theoretical concept."


# Augment the data
augmented_question, augmented_context = augment_data(question, context)

# Tokenize the augmented data
encoded_inputs = tokenizer(augmented_question, augmented_context, truncation=True, padding=True, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# Instantiate the custom classifier with contextual embeddings
model = CustomClassifierWithContextualEmbeddings(hidden_size=768)  # Assuming BERT-base


# Define the loss function
loss_function = nn.CrossEntropyLoss()

# Compute the start and end positions
start_logits, end_logits = model(input_ids, attention_mask)

# Generate target tensors for training (example)
start_positions = torch.tensor([1])
end_positions = torch.tensor([5])

# Compute the loss
start_loss = loss_function(start_logits, start_positions)
end_loss = loss_function(end_logits, end_positions)
total_loss = start_loss + end_loss

# Perform backpropagation and update the model parameters
total_loss.backward()



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:

from transformers import AdamW

# Set random seed for reproducibility (optional)
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

# Define training settings
batch_size = 16
learning_rate = 2e-5
num_epochs = 5

# Define the optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = num_epochs * len(input_ids) // batch_size
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate, total_steps=total_steps)

# Set the model in training mode
#model.train()


ValueError: Expected positive integer total_steps, but got 0

In [36]:
total_steps = num_epochs * (len(input_ids) // batch_size)

In [37]:
total_steps

0

In [33]:
batch_size

16

In [41]:
len(input_ids)

1

In [42]:
encoded_inputs

{'input_ids': tensor([[ 101, 2054, 2003, 1996, 3007, 1997, 2605, 1029, 1029,  102, 3000, 2003,
         1996, 3007, 1998, 2922, 2103, 1997, 2605, 1012,  103,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}